This notebook takes the generated text, cleans it as necessary, and performs a variety of embeddings.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gensim
import pandas
import sklearn

In [ ]:
import numpy as np

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en")

### Import Data and Clean

In [ ]:
data_d = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/quarterly-democrats.csv")
data_s = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/quarterly-socialism.csv")
data_c17 = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/2017-conservative.csv")
data_c18 = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/2018-conservative.csv")
data_c19 = pd.read_csv("/content/drive/My Drive/Final Project/Data/text-generation-results/2019-conservative.csv")

In [ ]:
#Socialism
for i in range(len(data_s)):
    seq_len = len(data_s.Prompt[i].split(" "))
    data_s.Result[i] = data_s.Result[i].replace("\n\n", " \n\n ")
    data_s.Result[i] = " ".join(data_s.Result[i].split(" ")[seq_len:])

#Democrats
for i in range(len(data_d)):
    seq_len = len(data_d.Prompt[i].split(" "))
    data_d.Result[i] = data_d.Result[i].replace("\n\n", " \n\n ")
    data_d.Result[i] = " ".join(data_d.Result[i].split(" ")[seq_len:])

#Conservative 2017
for i in range(len(data_c17)):
    seq_len = len(data_c17.Prompt[i].split(" "))
    data_c17.Result[i] = data_c17.Result[i].replace("\n\n", " \n\n ")
    data_c17.Result[i] = " ".join(data_c17.Result[i].split(" ")[seq_len:])

#Conservative 2018
for i in range(len(data_c18)):
    seq_len = len(data_c18.Prompt[i].split(" "))
    data_c18.Result[i] = data_c18.Result[i].replace("\n\n", " \n\n ")
    data_c18.Result[i] = " ".join(data_c18.Result[i].split(" ")[seq_len:])

#Conservative 2019
for i in range(len(data_c19)):
    seq_len = len(data_c19.Prompt[i].split(" "))
    data_c19.Result[i] = data_c19.Result[i].replace("\n\n", " \n\n ")
    data_c19.Result[i] = " ".join(data_c19.Result[i].split(" ")[seq_len:])


In [ ]:
# Add label column
data_s["label"] = "socialism"
data_d["label"] = "democrats"
data_c17["label"] = "conservative"
data_c18["label"] = "conservative"
data_c19["label"] = "conservative"

In [ ]:
# Only keep q1 and q2 for 2019 conservative
data_c19 = data_c19.iloc[0:540, :]

In [ ]:
#for loop below
dfs = ["data_c17", "data_c18", "data_c19", "data_d", "data_s"]

In [ ]:
#create quarter column for conservative data
data_c17['Quarter'] = 0
for i in range(len(data_c17)):
    if data_c17.Month[i] >= 1 and data_c17.Month[i] <= 3:
        data_c17.Quarter[i] = 1
    elif data_c17.Month[i] > 3 and data_c17.Month[i] <= 6:
        data_c17.Quarter[i] = 2
    elif data_c17.Month[i] > 6 and data_c17.Month[i] <= 9:
        data_c17.Quarter[i] = 3
    else:
        data_c17.Quarter[i] = 4
del data_c17['Month']

data_c18['Quarter'] = 0
for i in range(len(data_c18)):
    if data_c18.Month[i] >= 1 and data_c18.Month[i] <= 3:
        data_c18.Quarter[i] = 1
    elif data_c18.Month[i] > 3 and data_c18.Month[i] <= 6:
        data_c18.Quarter[i] = 2
    elif data_c18.Month[i] > 6 and data_c18.Month[i] <= 9:
        data_c18.Quarter[i] = 3
    else:
        data_c18.Quarter[i] = 4
del data_c18['Month']

data_c19['Quarter'] = 0
for i in range(len(data_c19)):
    if data_c19.Month[i] >= 1 and data_c19.Month[i] <= 3:
        data_c19.Quarter[i] = 1
    elif data_c19.Month[i] > 3 and data_c19.Month[i] <= 6:
        data_c19.Quarter[i] = 2
    elif data_c19.Month[i] > 6 and data_c19.Month[i] <= 9:
        data_c19.Quarter[i] = 3
    else:
        data_c19.Quarter[i] = 4
del data_c19['Month']

In [ ]:
data = data_d.append(data_s.append(data_c17.append(data_c18.append(data_c19))))

## D2V Two Dimensions

In [ ]:
import spacy
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import numpy as np

In [ ]:
prompts = data['Prompt'].unique()

In [ ]:
nlp = spacy.load('en')

#### Prompt 1

In [ ]:
data_1 = data[data.Prompt == prompts[1]]

In [ ]:
texts, article = [], []
for text in data_1['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_1_embeddings = data_1

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_1_embeddings["D2V_x"] = results_x
data_1_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_1_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
10,10,1,2017,"When it comes to data privacy, Facebook",hacked. \n\n The FBI has also been investiga...,democrats,0.200720,-0.217393
11,11,1,2017,"When it comes to data privacy, Facebook",the most vocal opponents of the data-mining pr...,democrats,-0.046209,-0.196830
12,12,1,2017,"When it comes to data privacy, Facebook",ones that have the most to lose. \n\n The pr...,democrats,0.114923,-0.144610
13,13,1,2017,"When it comes to data privacy, Facebook",problem with it. \n\n The company has been u...,democrats,-0.126268,-0.156822
14,14,1,2017,"When it comes to data privacy, Facebook",a private company that has a lot of control ov...,democrats,-0.186804,-0.151225
...,...,...,...,...,...,...,...,...
465,465,2,2019,"When it comes to data privacy, Facebook",and Google are the only two companies that hav...,conservative,-0.057625,0.082807
466,466,2,2019,"When it comes to data privacy, Facebook",is the only company that has a monopoly on dat...,conservative,-0.032545,-0.180477
467,467,2,2019,"When it comes to data privacy, Facebook","Google, and Twitter have all been guilty of th...",conservative,-0.140396,-0.015738
468,468,2,2019,"When it comes to data privacy, Facebook","Twitter, and Google have all been doing a good...",conservative,-0.175232,-0.255840


#### Prompt 0

In [ ]:
data_0 = data[data.Prompt == prompts[0]]

In [ ]:
texts, article = [], []
for text in data_0['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_0_embeddings = data_0

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_0_embeddings["D2V_x"] = results_x
data_0_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_0_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
0,0,1,2017,When it comes to data privacy,government is not the only entity that can col...,democrats,0.124954,-0.069945
1,1,1,2017,When it comes to data privacy,government is not the only one that can collec...,democrats,-0.073580,0.126548
2,2,1,2017,When it comes to data privacy,that can be done to protect your data. \n\n ...,democrats,0.080702,0.042353
3,3,1,2017,When it comes to data privacy,who are not happy with the way the government ...,democrats,-0.302438,0.046806
4,4,1,2017,When it comes to data privacy,that the government is not the only entity tha...,democrats,-0.035936,0.110617
...,...,...,...,...,...,...,...,...
455,455,2,2019,When it comes to data privacy,there are a lot of people who don't like the i...,conservative,0.041185,-0.010656
456,456,2,2019,When it comes to data privacy,it’s not the same as saying that you‘re not al...,conservative,-0.192676,-0.082316
457,457,2,2019,When it comes to data privacy,there are a lot of things that can be done to ...,conservative,-0.192475,-0.172880
458,458,2,2019,When it comes to data privacy,it’s a good thing. \n\n I think it's a shame t...,conservative,-0.243205,-0.405448


#### Prompt 2

In [ ]:
data_2 = data[data.Prompt == prompts[2]]

In [ ]:
texts, article = [], []
for text in data_2['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_2_embeddings = data_2

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_2_embeddings["D2V_x"] = results_x
data_2_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_2_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
20,20,1,2017,My view on Facebook is,"thing as a ""safe space"" for hate speech. \n\...",democrats,0.307393,-0.041559
21,21,1,2017,My view on Facebook is,time to try to find out what people are thinki...,democrats,0.300454,-0.140838
22,22,1,2017,My view on Facebook is,"to use the ""social media"" tool to find out wha...",democrats,0.138165,-0.193238
23,23,1,2017,My view on Facebook is,idea to post anything that isn't true. \n\n ...,democrats,-0.044331,0.204448
24,24,1,2017,My view on Facebook is,of misinformation out there. \n\n I'm not ...,democrats,0.474117,-0.079212
...,...,...,...,...,...,...,...,...
475,475,2,2019,My view on Facebook is,that it’s better to be a conservative than a l...,conservative,-0.054146,0.028939
476,476,2,2019,My view on Facebook is,that it’s a social media platform. It”s not a ...,conservative,0.264272,-0.256883
477,477,2,2019,My view on Facebook is,that it’s just a platform for people to post a...,conservative,0.170897,-0.287324
478,478,2,2019,My view on Facebook is,that it’s a private company and that they‘re f...,conservative,0.220588,-0.278540


#### Prompt 3

In [ ]:
data_3 = data[data.Prompt == prompts[3]]

In [ ]:
texts, article = [], []
for text in data_3['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_3_embeddings = data_0

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_3_embeddings["D2V_x"] = results_x
data_3_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_3_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
0,0,1,2017,When it comes to data privacy,government is not the only entity that can col...,democrats,-0.209694,-0.168932
1,1,1,2017,When it comes to data privacy,government is not the only one that can collec...,democrats,-0.218647,-0.169047
2,2,1,2017,When it comes to data privacy,that can be done to protect your data. \n\n ...,democrats,-0.116015,0.044761
3,3,1,2017,When it comes to data privacy,who are not happy with the way the government ...,democrats,-0.418130,-0.171567
4,4,1,2017,When it comes to data privacy,that the government is not the only entity tha...,democrats,-0.042808,0.220697
...,...,...,...,...,...,...,...,...
455,455,2,2019,When it comes to data privacy,there are a lot of people who don't like the i...,conservative,-0.362734,-0.150963
456,456,2,2019,When it comes to data privacy,it’s not the same as saying that you‘re not al...,conservative,0.082891,-0.002361
457,457,2,2019,When it comes to data privacy,there are a lot of things that can be done to ...,conservative,-0.279621,0.179616
458,458,2,2019,When it comes to data privacy,it’s a good thing. \n\n I think it's a shame t...,conservative,-0.173210,0.156899


#### Prompt 4

In [ ]:
data_4 = data[data.Prompt == prompts[4]]

In [ ]:
texts, article = [], []
for text in data_4['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_4_embeddings = data_4

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_4_embeddings["D2V_x"] = results_x
data_4_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_4_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
40,40,1,2017,Facebook's impact on society is,thing that the media is not as biased as they ...,democrats,-0.250805,-0.128175
41,41,1,2017,Facebook's impact on society is,"I'm not saying that Trump is a bad person, but...",democrats,-0.263725,0.023207
42,42,1,2017,Facebook's impact on society is,&gt;The problem is that the GOP is not a party...,democrats,-0.209572,0.088490
43,43,1,2017,Facebook's impact on society is,"I'm not saying that Trump is a bad person, but...",democrats,-0.315651,0.023309
44,44,1,2017,Facebook's impact on society is,&gt;The problem is that the GOP is not a party...,democrats,-0.243600,0.091794
...,...,...,...,...,...,...,...,...
495,495,2,2019,Facebook's impact on society is,undeniable. \n\n &amp;#x200B;\nI'm not saying...,conservative,-0.070198,0.208224
496,496,2,2019,Facebook's impact on society is,huge. \n\n &amp;#x200B;\nI'm not sure how you ...,conservative,0.198538,0.235822
497,497,2,2019,Facebook's impact on society is,so great that it's hard to imagine a country w...,conservative,0.012024,-0.011187
498,498,2,2019,Facebook's impact on society is,undeniable. \n\n The fact that the left is so...,conservative,-0.329100,-0.054811


#### Prompt 5

In [ ]:
data_5 = data[data.Prompt == prompts[5]]

In [ ]:
texts, article = [], []
for text in data_5['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_5_embeddings = data_5

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_5_embeddings["D2V_x"] = results_x
data_5_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_5_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
50,50,1,2017,Social media's impact on society is,also a social phenomenon that has been around ...,democrats,-0.012192,0.256591
51,51,1,2017,Social media's impact on society is,also includes the use of social networks and s...,democrats,-0.196303,0.248415
52,52,1,2017,Social media's impact on society is,\n\n The internet is a powerful tool for soci...,democrats,0.244070,0.098608
53,53,1,2017,Social media's impact on society is,\n\n The US has a history of using social med...,democrats,-0.248494,0.183039
54,54,1,2017,Social media's impact on society is,"\nI'm not saying that the media is bad, but I'...",democrats,-0.165472,-0.134368
...,...,...,...,...,...,...,...,...
505,505,2,2019,Social media's impact on society is,so great that it's hard to imagine a country w...,conservative,-0.247917,0.366400
506,506,2,2019,Social media's impact on society is,undeniable. \n\n &amp;#x200B;\nI'm not sure ho...,conservative,0.094441,0.234577
507,507,2,2019,Social media's impact on society is,undeniable. \n\n &amp;#x200B;\nI'm not saying...,conservative,-0.024124,0.226595
508,508,2,2019,Social media's impact on society is,undeniable. \n\n I'm not saying that the left...,conservative,0.070345,0.065305


#### Prompt 6

In [ ]:
data_6 = data[data.Prompt == prompts[6]]

In [ ]:
texts, article = [], []
for text in data_6['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_6_embeddings = data_6

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_6_embeddings["D2V_x"] = results_x
data_6_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_6_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
60,60,1,2017,The impact of social media on society is,are a powerful tool for social change. \n\n ...,democrats,-0.223333,0.216225
61,61,1,2017,The impact of social media on society is,a powerful tool for social change. \n\n The ...,democrats,-0.253880,0.225063
62,62,1,2017,The impact of social media on society is,networks are not the only tools that can be us...,democrats,-0.060859,0.279334
63,63,1,2017,The impact of social media on society is,"that social networking is bad, but it's not a ...",democrats,-0.025197,0.204400
64,64,1,2017,The impact of social media on society is,"that social networking is bad, but it's not a ...",democrats,-0.334338,-0.171390
...,...,...,...,...,...,...,...,...
515,515,2,2019,The impact of social media on society is,pretty clear. \n\n The left is trying to make...,conservative,-0.296993,-0.107555
516,516,2,2019,The impact of social media on society is,not limited to the US. \n\n The US is a countr...,conservative,-0.319851,0.090394
517,517,2,2019,The impact of social media on society is,pretty clear. \n\n &amp;#x200B;\nI'm not sure ...,conservative,-0.224407,0.173043
518,518,2,2019,The impact of social media on society is,pretty clear. \n\n I'm not saying that it's b...,conservative,-0.253109,-0.122163


#### Prompt 7

In [ ]:
data_7 = data[data.Prompt == prompts[7]]

In [ ]:
texts, article = [], []
for text in data_7['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_7_embeddings = data_7

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_7_embeddings["D2V_x"] = results_x
data_7_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_7_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
70,70,1,2017,Facebook's impact on democracy is,thing. \n\n I'm not saying that Trump is a b...,democrats,0.133593,-0.170104
71,71,1,2017,Facebook's impact on democracy is,remember that the DNC is not a party of the we...,democrats,-0.355462,0.286589
72,72,1,2017,Facebook's impact on democracy is,huge deal. \n\n I'm not saying that Trump is...,democrats,-0.348607,0.277355
73,73,1,2017,Facebook's impact on democracy is,"I'm not saying that Trump is a bad person, but...",democrats,-0.148179,0.023176
74,74,1,2017,Facebook's impact on democracy is,limited to the right. \n\n The right is not ...,democrats,-0.237752,0.223658
...,...,...,...,...,...,...,...,...
525,525,2,2019,Facebook's impact on democracy is,far greater than that of Facebook. \n\n &gt;Th...,conservative,-0.196822,0.172814
526,526,2,2019,Facebook's impact on democracy is,far greater than that of any other platform. \...,conservative,-0.246542,0.175896
527,527,2,2019,Facebook's impact on democracy is,undeniable. \n\n The left is so obsessed with...,conservative,0.144780,-0.118619
528,528,2,2019,Facebook's impact on democracy is,far greater than that of a single company. \n\...,conservative,-0.084868,0.205983


#### Prompt 8

In [ ]:
data_8 = data[data.Prompt == prompts[8]]

In [ ]:
texts, article = [], []
for text in data_8['Result']:
    text = text.replace('\n\n', '')
    doc = nlp(text)
    article = []
    for w in doc:
        # if it's not a stop word or punctuation mark, add it to our article!
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            # we add the lematized version of the word
            article.append(w.lemma_)

    texts.append(article)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]

In [ ]:
d2vmodel = Doc2Vec(documents, vector_size=2)

In [ ]:
w2vmodel = Word2Vec(
        texts,
        size=100,
        window=10,
        workers=10,
        iter=10)

In [ ]:
def create_vector(text, model, model_type=None):
    if model_type == "word2vec":
        vectors = []
        for word in text:
            try:
                vectors.append(model.wv[word])
            except KeyError:
                pass
        if len(vectors) > 0:
            return np.mean(vectors, axis=0)
    if model_type == "doc2vec":
        vector = model.infer_vector(text)
        return vector

In [ ]:
d2v_results = []
for text in texts:
  d2v = create_vector(text, d2vmodel, model_type="doc2vec")
  d2v_results.append(d2v)
d2v_results = [l.tolist() for l in d2v_results]

In [ ]:
# Add D2V results back into DF
data_8_embeddings = data_8

In [ ]:
results_x = []
for i in range(len(d2v_results)):
    results_x.append(d2v_results[i][0])
results_y = []
for i in range(len(d2v_results)):
    results_y.append(d2v_results[i][1])

In [ ]:
data_8_embeddings["D2V_x"] = results_x
data_8_embeddings["D2V_y"] = results_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_8_embeddings

,Unnamed: 0,Quarter,Year,Prompt,Result,label,D2V_x,D2V_y
80,80,1,2017,"When it comes to Facebook, the government should",and cannot post. \n\n I'm not saying that th...,democrats,-0.210335,0.082264
81,81,1,2017,"When it comes to Facebook, the government should",\n\n I'm not saying that the US government s...,democrats,0.157909,0.160140
82,82,1,2017,"When it comes to Facebook, the government should",access the site. \n\n The government has a r...,democrats,-0.204941,-0.103727
83,83,1,2017,"When it comes to Facebook, the government should",it. \n\n I'm not saying that the US governme...,democrats,-0.306095,0.092187
84,84,1,2017,"When it comes to Facebook, the government should",on their own. \n\n I'm not saying that the U...,democrats,-0.234720,0.192524
...,...,...,...,...,...,...,...,...
535,535,2,2019,"When it comes to Facebook, the government should",be able to ban it. \n\n The government shouldn...,conservative,-0.275640,-0.126885
536,536,2,2019,"When it comes to Facebook, the government should",not be able to control the content of its user...,conservative,-0.249792,0.045697
537,537,2,2019,"When it comes to Facebook, the government should",be able to regulate it. \n\n The government sh...,conservative,-0.273159,-0.137597
538,538,2,2019,"When it comes to Facebook, the government should",have the power to ban it. \n\n The government ...,conservative,-0.357010,-0.039970


## Combine Embedding Data

In [ ]:
final_df = data_0_embeddings.append(data_1_embeddings.append(data_2_embeddings.append(data_3_embeddings.append(data_4_embeddings.append(data_5_embeddings.append(data_6_embeddings.append(data_7_embeddings.append(data_8_embeddings))))))))

In [ ]:
final_df.to_csv("/content/drive/My Drive/Final Project/Data/Embeddings/embeddings-2d-d2v-all-subreddits.csv")